# Calibration of baseline control parameters

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

import os
import json
import copy
import itertools
from collections import Counter, defaultdict
import pandas as pd
import networkx as nx
import numpy as np
from multiprocessing import cpu_count, Pool

from lib.graph_generation import make_ebola_network
from lib.dynamics import SimulationSIR, PriorityQueue
from lib.dynamics import sample_seeds
from lib.settings import DATA_DIR
from lib import metrics

---

## 1. Set simulation parameters

Set simulation period

In [3]:
start_day_str = '2014-01-01'
end_day_str = '2014-04-01'
max_timedelta = pd.to_datetime(end_day_str) - pd.to_datetime(start_day_str)
max_days = max_timedelta.days

Set SIR infection and recovery rates

In [4]:
beta = 1 / 15.3
delta = 1 / 11.4

gamma = beta
rho = 0.0

Set the network parameters.

In [5]:
n_nodes = 8000
p_in = 0.01
p_out = {
           'Guinea': 0.00215,
          'Liberia': 0.00300, 
     'Sierra Leone': 0.00315, 
    'inter-country': 0.0019
}

Set the control parameters.

In [6]:
DEFAULT_POLICY_PARAMS = {
    # SOC parameters
    'eta':   1.0,          # SOC exponential decay
    'q_x':   None,       # SOC infection cost
    'q_lam':   1.0,        # SOC recovery cost
    'lpsolver': 'cvxopt',  # SOC linear progam solver
    
    # Scaling of baseline methods
    'TR' : None,
    'MN' : None,
    'LN' : None,
    'LRSR' : None,
    'MCM' : None,
    'front-loading': {  # Front-loading parameters (will be set for each simulation after the SOC run)
        'max_interventions': None,
        'max_lambda': None
    }
}


---

## 2. Run calibration



Define the range of $q_x$ to experiment with.

In [7]:
q_x_range = [1.0, 10.0, 25.0, 50.0, 100.0, 250.0, 500.0, 750.0, 1000.0]

Define the worker function to run several simulation on a given network

In [8]:
def worker(policy, policy_params, n_sims):
    graph = make_ebola_network(n_nodes=n_nodes, p_in=p_in, p_out=p_out)
    print(f'graph: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges')

    init_event_list = sample_seeds(graph, delta=delta, method='data', 
                                   max_date=start_day_str, verbose=False)

    res_dict = {'max_u': list(), 'n_tre': list()}

    for sim_idx in range(n_sims):
        print(f"\rSim {sim_idx+1}/{n_sims}", end="")
         
        sir_obj = SimulationSIR(graph, beta=beta, delta=delta, gamma=gamma, rho=rho, verbose=True)
        sir_obj.launch_epidemic(
            init_event_list=init_event_list,
            max_time=max_days, 
            policy=policy,
            policy_dict=policy_params
        )
        
        res_dict['max_u'].append(sir_obj.max_total_control_intensity)
        res_dict['n_tre'].append(sir_obj.is_tre.sum())
    
    return res_dict

Run the first simulations with SOC policy.

In [9]:
res_list = list()

num_nets = 5
num_sims = 5

for i, q_x in enumerate(q_x_range):
    print(f'=== q_x {i+1} / {len(q_x_range)}')
    
    for net_idx in range(num_nets):
        print(f'--- Network {net_idx+1} / {num_nets}')

        policy_params = copy.deepcopy(DEFAULT_POLICY_PARAMS)
        policy_params['q_x'] = q_x

        d = worker(policy='SOC', policy_params=policy_params, n_sims=num_sims)
    
        res_list.append({
            'policy': 'SOC',
            'q_x': q_x,
            'net_idx': net_idx,
            'max_u': d['max_u'],
            'n_tre': d['n_tre']
        })
        
res_df = pd.DataFrame(res_list)
res_df.to_csv('baseline-calibration-soc.csv', index=False)

=== q_x 1 / 9
--- Network 1 / 5
graph: 7456 nodes, 15962 edges
31.96 days elapsed | 6726 sus., 423 inf., 307 rec., 20 tre (4.73% of inf) | max_u 5.08e+00

KeyboardInterrupt: 

In [113]:
policy_params = copy.deepcopy(DEFAULT_POLICY_PARAMS)
policy_params['TR'] = 0.03

res_dict_baseline = worker(policy='TR', policy_params=policy_params, n_sims=10)
res_dict_baseline

Epidemic stopped after 31.00 days | 657 sus., 48 inf., 67 rec., 20 tre (41.67% of inf) | I(q): 222 R(q): 48 T(q): 86 |q|: 354
Epidemic stopped after 31.00 days | 708 sus., 32 inf., 32 rec., 7 tre (21.88% of inf) | I(q): 120 R(q): 33 T(q): 52 |q|: 203
Epidemic stopped after 31.00 days | 762 sus., 1 inf., 9 rec., 5 tre (500.00% of inf) | I(q): 20 R(q): 1 T(q): 1 |q|: 20
Epidemic stopped after 31.00 days | 735 sus., 10 inf., 27 rec., 4 tre (40.00% of inf) | I(q): 85 R(q): 11 T(q): 27 |q|: 121
Epidemic stopped after 31.00 days | 667 sus., 34 inf., 71 rec., 25 tre (73.53% of inf) | I(q): 182 R(q): 35 T(q): 66 |q|: 281
Epidemic stopped after 31.00 days | 736 sus., 11 inf., 25 rec., 3 tre (27.27% of inf) | I(q): 90 R(q): 12 T(q): 29 |q|: 129
Epidemic stopped after 31.00 days | 519 sus., 94 inf., 159 rec., 42 tre (44.68% of inf) | I(q): 377 R(q): 95 T(q): 175 |q|: 645
Epidemic stopped after 31.00 days | 699 sus., 20 inf., 53 rec., 10 tre (50.00% of inf) | I(q): 116 R(q): 21 T(q): 51 |q|: 186
E

{'max_u': [3.4499999999999997,
  1.92,
  0.3,
  1.11,
  3.1500000000000004,
  1.08,
  7.590000000000001,
  2.19,
  0.21,
  2.9999999999999996],
 'n_tre': [20, 7, 5, 4, 25, 3, 42, 10, 3, 13]}